# Deterministic Uncertainty Sets - Direct comparison constraints, budget uncertainty

In [1]:
import cvxpy as cp
import scipy as sc
from sklearn import datasets
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import torch
import pandas as pd
import lropt
import matplotlib.pyplot as plt
RTOL = 1e-04
ATOL = 1e-04
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

We solve the problem
$$
\begin{array}{ll}
\text{minimize} & \tau\\
\text{subject to}  & -(1/2)\sum_{i=1}^n (u^TP_iu)x_i  \leq  \tau\\
& \mathbf{1}^Tx = 3\\
& x \geq 0, \quad x \leq 1
\end{array}
$$
where each $P_i\in \mathbf{R}^{n\times n}$ is a symmetric positive definite matrix, and $u$ is the uncertain parameter from an ellipsoidal uncertainty set. 

In [2]:
m = 5
# formulate uncertainty set
n = 5
u = lropt.UncertainParameter(n,
                             uncertainty_set=lropt.Ellipsoidal(p=2, rho=0.5))
# formulate cvxpy variables
x_r = cp.Variable(n)
t = cp.Variable()

# formulate problem constants
P = {}
P_inv = {}
for i in range(n):
    P[i] = datasets.make_spd_matrix(n, random_state=i)
    P_inv[i] = sc.linalg.sqrtm(np.linalg.inv(P[i]))

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.sum([-0.5*lropt.quad_form(u, P[i]*x_r[i]) for i in range(n)]) <= t]
constraints += [cp.sum(x_r)==4]
constraints += [x_r >= 0.6, x_r <= 1]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -2.0631492961158832e-10 
LRO x:  [0.79877983 0.80138279 0.80072783 0.79878798 0.80032158]


We compare this with the direct reformulation,
\begin{equation}
	\begin{array}{lll}
		&\text{minimize} & \tau \\
		&\text{subject to} & (1/2)\sum_{i=1}^n (z_i^TP_i^{-1}z_i)/x_i + \rho\|z\|_2 \le \tau\\
		&& \sum_{i=1}^n z_i = z,\\
&& \mathbf{1}^Tx = 3\\
&& x \geq 0, \quad x \leq 1
	\end{array}
\end{equation}
where we have $\gamma \in \mathbf{R}^m, z \in \mathbf{R}^n, z_i \in \mathbf{R}^n$, $i = 1,\dots,n$, as auxiliary variables. 

In [3]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
t = cp.Variable()
z = cp.Variable(n)
y = cp.Variable((n,n))
gam = cp.Variable(n)

# formulate objective
objective = cp.Minimize(t)

# formulate constraints
constraints = [cp.sum([cp.quad_over_lin(y[i]@P_inv[i],2*x_cvxpy[i]) for i in range(n)]) + 0.5*cp.norm(z,2) <= t]
constraints += [cp.sum(y, axis = 1) == z]
constraints += [cp.sum(x_cvxpy) == 4]
constraints += [x_cvxpy >= 0.6, x_cvxpy <= 1]

# formulate problem 
prob_cvxpy = cp.Problem(objective, constraints)
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -2.1183732382563087e-10 
Cvxpy x:  [0.79881653 0.80133787 0.80070692 0.79882447 0.80031422]
